In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '_jupyterNotebooks'))
from weis_workshop import *
import openmdao.api as om
import plotly.express as px
import pickle

from openfast_io.FAST_writer import InputWriter_OpenFAST
from openfast_io.FileTools import compare_fst_vt, load_yaml

import pandas as pd


Running in local environment


/kfs2/projects/weis/mchetan/weis-workshop/tools/rosco/rosco/toolbox/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
rec_data = load_OMsql('/projects/weis/mchetan/weis-workshop/workshopFiles/stage-2-controller/outputs/log_opt.sql')

# rec_data = om.CaseReader('outputs_of/log_opt.sql')



loading /projects/weis/mchetan/weis-workshop/workshopFiles/stage-2-controller/outputs/log_opt.sql


driver
rank0:NLopt_LN_COBYLA|0
rank0:NLopt_LN_COBYLA|1
rank0:NLopt_LN_COBYLA|2
rank0:NLopt_LN_COBYLA|3
rank0:NLopt_LN_COBYLA|4
rank0:NLopt_LN_COBYLA|5
rank0:NLopt_LN_COBYLA|6
rank0:NLopt_LN_COBYLA|7
rank0:NLopt_LN_COBYLA|8
rank0:NLopt_LN_COBYLA|9


In [ ]:
rec_data.keys()
print(f'tune_rosco_ivc.Kp_float: {rec_data["tune_rosco_ivc.Kp_float"][-1]}')
print(f'tune_rosco_ivc.omega_pc: {rec_data["tune_rosco_ivc.omega_pc"][-1]}')
print(f'tune_rosco_ivc.ptfm_freq: {rec_data["tune_rosco_ivc.ptfm_freq"][-1]}')
print(f'tune_rosco_ivc.zeta_pc: {rec_data["tune_rosco_ivc.zeta_pc"][-1]}')

dict_keys(['aeroelastic.DEL_TwrBsMyt', 'aeroelastic.rotor_overspeed', 'sse_tune.tune_rosco.PC_Ki', 'sse_tune.tune_rosco.PC_Kp', 'tune_rosco_ivc.Kp_float', 'tune_rosco_ivc.omega_pc', 'tune_rosco_ivc.ptfm_freq', 'tune_rosco_ivc.zeta_pc'])

In [ ]:
# oepning a pickle file
with open('/projects/weis/mchetan/weis-workshop/workshopFiles/stage-3-semisub/outputs_of/openfast_runs/rank_0/iteration_46/fst_vt.p', 'rb') as f:
    stats = pickle.load(f)

with open('/projects/weis/mchetan/weis-workshop/workshopFiles/stage-3-semisub/outputs_of/openfast_runs/rank_0/iteration_48/fst_vt.p', 'rb') as f:
    stats_2 = pickle.load(f)

caseMat = pd.read_csv('outputs_of/openfast_runs/rank_0/case_matrix_combined.txt', sep='\s+', header = 0)

In [ ]:
caseMat['(InflowWind,FileName_BTS)'][1]

In [ ]:
compare_fst_vt(stats, stats_2, ignoreVars = None, removeFileRef=True, removeArrayProps=True, print_diff=True)

In [ ]:
writer = InputWriter_OpenFAST()
writer.FAST_namingOut = 'scratch'    #Master FAST file
writer.FAST_runDirectory = '_scratch' #Output directory

stats['InflowWind']['FileName_BTS'] = caseMat['(InflowWind,FileName_BTS)'][1]
stats['Fst']['TMax'] = caseMat['(Fst,TMax)'][1]
stats['Fst']['TStart'] = caseMat['(Fst,TStart)'][1]
writer.fst_vt = stats

writer.execute()

In [ ]:
with open('/projects/weis/mchetan/weis-workshop/workshopFiles/stage-3-semisub/outputs_of/openfast_runs/rank_0/iteration_46/timeseries/stage-3-semisub_of_0_20.p', 'rb') as f:
    ts_46 = pickle.load(f)

In [ ]:
ts_46

In [ ]:
px.line(ts_46, x='Time', y=['RotSpeed', 'BldPitch1'], title='Speeds vs Time')